# Proje 1 - Pandas ile Veri Analizi
Aşağıda belirtilen soruların cevaplarını, ilgili boş hücrelere yazınız ve adı öğrenci numaralarından (proje, 1 ve/veya 2 kişi yapılabilir) oluşan nihai Jupyter Notebook (.jpynb) dosyasını, 03.04.2018 Salı geceyarısına kadar http://sorubank.ege.edu.tr/~moodle web sitesindeki ilgili ders sayfasını kullanılarak sisteme yükleyiniz. 04.04.2018 Çarşamba günü ders saatlerinde sınıfta proje kontrolü yapılacaktır, kontrol sırasında projeyi yapan herkesin bulunması gereklidir. Herhangi bir sorunuz olursa, http://sorubank.ege.edu.tr/~moodle web sitesindeki ilgili ders sayfasında bu proje ile ilgili açılan forumu kullanınız.

### Soru-1 (15 puan)
### Ülkelere ilişkin kişi başı gayri safi yurt içi hasıla verilerini alın:
Birleşmiş Milletler İnsani Gelişmişlik Raporları (Human Deveoplemen Reports) sitesinde, farklı boyutlar altında çeşitli göstergeleri içeren, ülkelerin 1990-2015 yılları arasındaki İnsani Gelişmişlik Verileri (http://hdr.undp.org/en/data) bulunmaktadır. Bu verilerden "Income/composition of resources" boyutu altında yer alan, "Gross domestic product (GDP) per capita" (kişi başı gayri safi yurt içi hasıla) göstergesine ilişkin verileri indirin ("Gross domestic product (GDP) per capita (2011 PPP $).csv"). Bu dosyadan verileri yükleyin ve bir DataFrame oluşturun.

### Veri Temizleme&Düzenleme:
```
1- Kodlamayı, ISO-8859-1 olarak ayarlayın (read_csv fonksiyonu içindeki encoding parametresi ile)
2- Dosyanın ilk satırını atlayın
3- Son satırı silin
4- Boş sütunları (bütün değerleri 'NaN' olan sütunları) silin
5- Herhangi bir verisi boş ('NaN') olan ülkeleri silin
6- Country sütununda bulunan değerlerin sonundaki ekstra boşlukları silin
7- Country sütunundaki ülke isimlerinin bazılarını aşağıdaki şekilde yeniden adlandırın:
    'Korea (Republic of)':'South Korea',
    'Venezuela (Bolivarian Republic of)':'Venezuela',
    'The former Yugoslav Republic of Macedonia':'Macedonia',
    'Hong Kong, China (SAR)':'Hong Kong'
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Ahmet Faruk Aktaş - Mertcan Takıl
gdp = pd.DataFrame(pd.read_csv('Gross domestic product (GDP) per capita (2011 PPP $).csv', skiprows=1, encoding = 'ISO-8859-1'))
gdp = gdp.drop(len(gdp)-1)
gdp = gdp.dropna(axis=1, how='all')
gdp = gdp.dropna(axis=0, how='any')
gdp['Country'] = gdp['Country'].str.strip()
gdp['Country'] = gdp['Country'].replace({'Korea (Republic of)':'South Korea', 'Venezuela (Bolivarian Republic of)':'Venezuela', 'The former Yugoslav Republic of Macedonia':'Macedonia', 'Hong Kong, China (SAR)':'Hong Kong'})

### Ülkelere ilişkin coğrafi verileri alın:
Geonames veri tabanından ülkelere ilişkin coğrafi verileri içeren dosyadan (http://download.geonames.org/export/dump/countryInfo.txt) verileri yükleyin ve başka bir DataFrame daha oluşturun.

### Veri Temizleme&Düzenleme:
```
1- Dosyanın ilk 50 satırını atlayın
2- Son satırı silin
3- '#ISO' sütununun ismini 'CountryCode' olarak değiştirin
4- 'ISO3', 'ISO-Numeric', 'fips', 'EquivalentFipsCode', 'Postal Code Format', 'Postal Code Regex' ve 'geonameid' sütunlarını silin
5- Kıta (Continent) değeri boş ('NaN') olan ülkelerin kıta değerini 'NA' (North America) olarak değiştirin
```

In [ ]:
ci = pd.DataFrame(pd.read_csv("countryInfo.txt", sep="\t", header=None, skiprows=50))
ci = ci.drop(len(ci)-1)
ci = ci.replace({'#ISO':'CountryCode'})
ci = ci.rename(columns=ci.iloc[0])
ci = ci.drop(0)
ci = ci.drop(columns = {'ISO3', 'ISO-Numeric', 'fips', 'EquivalentFipsCode', 'Postal Code Format', 'Postal Code Regex', 'geonameid'})
ci['Continent'] = ci['Continent'].replace({np.NaN:'NA'})

### Verilerinizi birleştirin:
Ülke adı ('Country') sütunlarının kesişimini kullanarak bu iki veri kümesini birleştirin ve 'Country' sütununu index olarak beirleyin.

In [ ]:
intersection = pd.merge(gdp, ci, on = 'Country')
intersection = intersection.rename(intersection['Country'])
intersection = intersection.drop(columns = {'Country'})

### Soru-2  (10 puan)
Ülkelerin, 1990-2015 yıllarını kapsayan 'GDP ortalama' değerlerini bulun ve veri setine yeni bir sütun olarak ekleyin.

In [ ]:

intersection['GDP-ortalama'] = intersection.loc[:, '1990':'2015'].mean(axis = 1)

Ülkelerin GDP ortalamalarını gösteren bir histogram (hist) çizin.

In [ ]:
intersection.hist(column='GDP-ortalama')
plt.show()

GDP ortalaması en yüksek ilk 10 ülkeyi gösteren bir sütun (bar) grafik çizin.

In [ ]:
intersection.nlargest(10, columns='GDP-ortalama').loc[:, 'GDP-ortalama'].plot.bar()
plt.show()

### Soru-3  (5 puan)
Her ülkenin kişi başına düşen yüzey alanını hesaplayınız ve kişi başına düşen yüzey alanı en küçük olan ilk 20 ülkeyi gösteren bir sütun grafik çizin.

In [ ]:
intersection['Area(in sq km)'] = pd.to_numeric(intersection['Area(in sq km)'])
intersection['Population'] = pd.to_numeric(intersection['Population'])
(intersection['Area(in sq km)']/intersection['Population']).nsmallest(20).plot.bar()
plt.show()

### Soru-4  (10 puan)
Kıtalara göre 'GDP ortalama' değerlerini gösteren bir sütun grafik çizin.

In [ ]:
intersection[['GDP-ortalama','Continent']].groupby('Continent').plot.barh()
plt.show()

Kıtalara göre toplam nüfus ve toplam alan dağılımlarını gösteren pasta (pie) grafikleri içeren bir şekil (figure) çizin. 

In [ ]:
intersection.groupby('Continent').plot.barh(x = 'Continent', y = 'GDP-ortalama')
intersection.groupby('Continent')['Population'].sum().plot.pie()
plt.show()

### Soru-5 (8 puan)
Veri setinden Türkiye'nin komşularını bulun, bu ülkelerin ve Türkiye'nin GDP ortalamalarını gösteren bir sütun grafik çizin.

In [ ]:
intersection.loc[intersection['CountryCode'].isin(intersection.loc['Turkey', 'neighbours'].split(',')), 'GDP-ortalama'].plot.bar()
plt.show()

### Soru-6 (12 puan) 
Her ülke için o ülkenin komşu ülkelerinin ortalama GDP değerlerinin ortalamasını hesaplayarak yeni bir seri oluşturunuz (eğer bir ülkenin komuşusu yoksa kendi GDP ortalamasını alınız) ve komşuları ile en fazla ortalama GDP farkı olan ilk 20 ülkeyi listeleyin.

In [ ]:
nbsmeans = pd.Series()
for index, row in intersection.iterrows():
    nbss = intersection.loc[row.name, 'neighbours']
    if pd.isnull(nbss):
        nbsmeans = nbsmeans.append(pd.Series([row['GDP-ortalama']]))
    else:
        nbsmeans = nbsmeans.append(pd.Series([intersection.loc[intersection['CountryCode'].isin(nbss.split(',')), 'GDP-ortalama'].mean()]))

print(intersection['GDP-ortalama'].subtract(pd.DataFrame(nbsmeans).set_index(intersection.index.values).rename(columns={0:'nbs'})['nbs']).abs().nlargest(20))

### Soru-7 (5 puan)
Ülkelerin yüzey ölçümü ile nüfusları arasındaki ilişkiyi gösteren bir dağılım (scatter) grafiği çizin. Baloncuklara ülke isimlerini ekleyin.

In [ ]:
intersection.plot.scatter(x='Area(in sq km)',y='Population',s=50,alpha=0.5,color='red')

### Soru - 8 (10 puan) 
İnsani gelişmişlik indeksi en yüksek 20 ülkenin, insani gelişmişlik indeksi ile ortalama GDP değerleri arasındaki ilişkiyi gösteren bir dağılım (scatter) grafiği çizin. Baloncukların büyüklüklerini, ortalama GDP değeriyle orantılayın. Baloncuklara ülke isimlerini ekleyin. Baloncukları kıtalara göre renklendirin.

In [ ]:
dizi = intersection['HDI Rank (2015)'].nsmallest(20)
hdi = []
for i in intersection['HDI Rank (2015)']:
    hdi.append(i)
gdp = []
for j in intersection['GDP-ortalama']:                  
    gdp.append(j)
gdp_size = []
for c in gdp:
    gdp_size.append(c/75)
plt.scatter(x=hdi,y=gdp,s=gdp_size,color='red',alpha=0.5)
plt.xlabel('HDI Rank (2015)')
plt.ylabel('GDP-ortalama')
plt.show()

### Soru 9 (10 puan)
1990 yılı ile 2015 yılı arasındaki GDP değeri değişimi en yüksek olan 15 ülkenin (1) 1990 ve 2015 GDP değerlerini, (2) fark  değerlerini gösteren sütun grafiklerini içeren bir şekil (figure) çizin.

In [ ]:
newdf2=pd.DataFrame()
newdf2['GDPdif']=intersection.loc[:,'1990'].sub(intersection.loc[:,'2015']).abs().nlargest(15)
for index,row in newdf2.iterrows():
    newdf2.loc[row.name,'1990'] = intersection.loc[row.name,'1990']                             
    newdf2.loc[row.name, '2015'] = intersection.loc[row.name, '2015']

newdf2.plot.barh()
plt.show()

### Soru 10 (5 puan)
Güney Amerika ('SA') ülkelerinin 1990'dan 2015'e kadar olan GDP değişimlerini yıllara göre gösteren bir çizgi (line) grafik çizin.  

In [ ]:
newdf = intersection[intersection.Continent=='SA']
newdf.loc[:,'1990':'2015'].plot.line()                      
plt.show()

### Soru 11 (10 puan)
Dünya'da en çok sayıda ülkede konuşulan ilk 10 dili (bir dilin ülkelere göre yerelleşmiş türlerini, aynı dil olarak kabul edin) ve konuşuldukları ülke sayılarını gösteren bir sütun grafik çizin.

In [ ]:
langpopularity = {}
for index, row in intersection.iterrows():
    ls = intersection.loc[row.name, 'Languages'].split(',')
    ls = [i.split('-', 1)[0] for i in ls]
    for l in ls:
        e = langpopularity.get(l)
        if e is None:
            langpopularity[l] = 1
        else:
            langpopularity[l] += 1